In [1]:
import json, numpy as np, pandas as pd, os

In [2]:
FOLDER = './results/reports/'
# this notebook creates the weighted average of results from each fold of every up-sampling methods

In [3]:
all_labels = ['BENIGN', 'DoS Hulk', 'SSH-Patator', 'PortScan', 'DoS GoldenEye', 'DDoS', 'Heartbleed', 'Web Attack Brute Force', 'FTP-Patator', 
              'Web Attack XSS', 'DoS slowloris', 'Infiltration', 'Bot', 'Web Attack Sql Injection', 'DoS Slowhttptest']

for f in os.listdir(FOLDER):
    report_file = os.path.join(FOLDER, f)
    if 'reports.json' in report_file:
        print('{} is being processed... '.format(report_file))

        report = json.loads(open(report_file, 'r').read())

        results  = []
        acc      = []
        macro    = []
        weighted = []
        for k in list(report.keys())[:-1]:
            result = []
            for l in report[k]:
                line = l.split()
                if len(line) > 0 and line[0] in ''.join(list(map(lambda x: str(x), range(15)))):
                    result.append(list(map(lambda x: round(float(x),6), line[1:-1])))
                if len(line) > 0 and (line[0] == 'accuracy'):
                    acc.append(list(map(lambda x: round(float(x),6), line[1:-1])))
                elif len(line) > 0 and (line[0] == 'macro'):
                    macro.append(list(map(lambda x: round(float(x),6), line[2:-1])))
                elif len(line) > 0 and (line[0] == 'weighted'):
                    weighted.append(list(map(lambda x: round(float(x),6), line[2:-1])))
            results.append(result)
        results  = sum(np.array(results)) / 5.0
        acc      = sum(np.array(acc)) / 5.0
        macro    = sum(np.array(macro)) / 5.0
        weighted = sum(np.array(weighted)) / 5.0

        overall = list(map(lambda x: np.concatenate([x[1], [all_labels[x[0]]]], axis=0), enumerate(results)))
        overall.append(['-', '-', '-', '-'])
        overall.append(['-', '-', acc[0], 'Accuracy'])
        overall.append(np.concatenate([macro, ['Macro Avg']], axis=0))
        overall.append(np.concatenate([weighted, ['Weighted Avg']], axis=0))

        df = pd.DataFrame(overall, columns=['precision', 'recall', 'f1-score', 'label'], dtype=np.float)
        df['precision'] = df['precision'].apply(lambda x: x[:8])
        df['recall']    = df['recall'].apply(lambda x: x[:8])
        df['f1-score']  = df['f1-score'].apply(lambda x: str(x)[:8])
        df['time']      = report['time'] / 60
        df['time'][:18] = ''
        df.to_csv('{}_overall.csv'.format(report_file.split('.json')[0]))

./results/reports/classifier_raw_reports.json is being processed... 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
